<a href="https://colab.research.google.com/github/Pau789/sentimiento-clima-reddit/blob/main/An%C3%A1lisis_de_sentimientos_y_t%C3%B3picos_en_Reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Explicación del Proyecto: Análisis de Sentimiento y Tópicos en Posts de Reddit sobre Cambio Climático

1.   Elemento de lista
2.   Elemento de lista

Reddit sobre Cambio Climático**

Este proyecto tiene como objetivo analizar publicaciones de Reddit relacionadas con el cambio climático. Utilizamos técnicas de procesamiento de lenguaje natural (NLP) para realizar análisis de sentimiento, clustering de textos y análisis de tópicos. A continuación, se explica cada parte del proyecto paso a paso.

---

#### **1. Configuración del Entorno**

Primero, instalamos e importamos las bibliotecas necesarias:

- **PRAW**: Para interactuar con la API de Reddit y obtener publicaciones.
- **Pandas**: Para manejar los datos en formato de DataFrame.
- **Spark**: Para procesamiento distribuido de datos.
- **Transformers (Hugging Face)**: Para análisis de sentimiento.
- **Sentence Transformers**: Para generar embeddings de texto.
- **Scikit-learn**: Para clustering y análisis de tópicos.

---

#### **2. Configuración de la API de Reddit**

Usamos PRAW para conectarnos a la API de Reddit. Se especifican las credenciales de la aplicación Reddit (`client_id`, `client_secret`, y `user_agent`). Estas credenciales son necesarias para autenticarse y acceder a los datos de Reddit.

---

#### **3. Definición de Términos Relacionados con el Cambio Climático**

Creamos una lista de términos clave relacionados con el cambio climático, como "climate change", "global warming", "sustainability", entre otros. Estos términos se utilizan para filtrar las publicaciones de Reddit y asegurarnos de que solo se analicen aquellas que están relacionadas con el tema.

---

#### **4. Extracción de Posts de Reddit**

Definimos una función para obtener publicaciones de un subreddit específico (`climatechange` en este caso). La función filtra las publicaciones según los términos definidos y extrae información relevante, como el título, el texto, la fecha, los upvotes y el número de comentarios. Si no se encuentran publicaciones que coincidan con los filtros, se muestra un mensaje indicando que no hay resultados.

---

#### **5. Análisis de Sentimiento con Hugging Face**

Utilizamos un modelo preentrenado de Hugging Face para analizar el sentimiento de los textos. El modelo clasifica cada texto como "positivo", "negativo" o "neutral". Esta función se aplica a cada publicación para determinar el sentimiento predominante.

---

#### **6. Clustering y Análisis de Tópicos**

Realizamos dos tipos de análisis adicionales:

1. **Clustering de Embeddings**: Utilizamos Sentence Transformers para generar embeddings de los textos y luego aplicamos el algoritmo KMeans para agrupar las publicaciones en clusters. Esto nos permite identificar grupos de publicaciones que tratan temas similares.

2. **Análisis de Tópicos con LDA**: Aplicamos Latent Dirichlet Allocation (LDA) para identificar los tópicos principales en las publicaciones. LDA es una técnica que permite descubrir temas ocultos en un conjunto de documentos.

---

#### **7. Función Principal**

Integramos todas las funciones anteriores en una función principal que ejecuta el flujo completo:

1. **Obtención de datos**: Se extraen las publicaciones de Reddit filtradas por los términos de cambio climático.
2. **Preprocesamiento**: Se tokenizan y limpian los textos utilizando Spark.
3. **Análisis de sentimiento**: Se clasifica el sentimiento de cada publicación.
4. **Clustering y análisis de tópicos**: Se agrupan las publicaciones y se identifican los tópicos principales.

---

#### **8. Ejecución**

Para ejecutar el proyecto, simplemente se llama a la función principal. Esto inicia el proceso de obtención de datos, análisis de sentimiento, clustering y análisis de tópicos.

---

#### **Resultados Esperados**

- **Posts Filtrados**: Se mostrarán los primeros 5 posts filtrados.
- **Preprocesamiento**: Se mostrarán las palabras tokenizadas y limpias.
- **Análisis de Sentimiento**: Se mostrará el sentimiento asociado a cada texto.
- **Clustering**: Se mostrarán los clusters generados y ejemplos de textos en cada cluster.
- **Tópicos**: Se mostrarán los tópicos principales identificados con LDA.

---

In [ ]:
!pip install praw findspark pyspark transformers torch vaderSentiment sentence-transformers scikit-learn

In [ ]:
import praw
import pandas as pd
from datetime import datetime
import findspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Configurar Spark (Google Colab ya soporta findspark)
findspark.init()
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

# Configurar Reddit API con PRAW (sin async)
reddit = praw.Reddit(client_id='t9a1-UEyorzFQLVNtasB9Q',
                     client_secret='LzJe_WUjUZvNPDC30AFJzzAwmxQd9Q',
                     user_agent='WeatherMood')

# Lista ampliada de términos relacionados con cambio climático
terminos_cambio_climatico = [
    'climate change', 'global warming', 'environmental alterations',
    'extreme weather', 'hurricanes', 'floods', 'droughts', 'glacier melting',
    'extreme temperatures', 'sea level rise', 'wildfires', 'deforestation',
    'carbon emissions', 'greenhouse gases', 'sustainability', 'climate crisis',
    'carbon footprint', 'renewable energy', 'solar energy', 'wind energy',
    'fossil fuels', 'energy transition', 'climate adaptation', 'climate action',
    'biodiversity loss', 'ocean acidification', 'carbon neutral', 'climate mitigation',
    'air pollution', 'water scarcity', 'ecosystem collapse', 'species extinction',
    'green economy', 'electric vehicles', 'net zero emissions', 'plastic pollution',
    'climate refugees', 'food security', 'reforestation', 'sustainable development',
    'circular economy', 'pollution reduction', 'carbon pricing', 'eco-friendly'
]

# Función para extraer posts de Reddit filtrados por términos de cambio climático
def obtener_posts_filtrados(subreddit, limite=100):
    try:
        posts = []
        subreddit = reddit.subreddit(subreddit)
        for post in subreddit.new(limit=limite):  # Obtener publicaciones nuevas
            post_text = post.selftext.lower() + " " + post.title.lower()  # Combina título y texto en minúsculas
            if any(termino in post_text for termino in terminos_cambio_climatico):
                post_dict = {
                    'Título': post.title,
                    'Texto': post.selftext if post.selftext else "No text",
                    'Fecha': datetime.fromtimestamp(post.created_utc),
                    'Upvotes': post.score,
                    'Número de Comentarios': post.num_comments,
                    'Enlace': post.url
                }
                posts.append(post_dict)
        df_posts = pd.DataFrame(posts)
        if df_posts.empty:
            print("No se encontraron posts que coincidan con los filtros.")
        return df_posts
    except Exception as e:
        print(f"Error al obtener los posts: {e}")
        return None

# Cargar el pipeline de análisis de sentimiento de Hugging Face
sentiment_pipeline = pipeline("sentiment-analysis")

# Función para analizar sentimiento usando el pipeline de Hugging Face
def analizar_sentimiento_transformers(texto):
    try:
        resultado = sentiment_pipeline(texto)
        label = resultado[0]['label']
        if label.upper() == "POSITIVE":
            return "positivo"
        elif label.upper() == "NEGATIVE":
            return "negativo"
        else:
            return "neutral"
    except Exception as e:
        return "neutral"

# Función para analizar tópicos y clustering de posts
def analyze_clustering_and_topics(df_posts):
    texts = df_posts["Texto"].tolist()

    # --- Clustering de Embeddings ---
    print("Generando embeddings con SentenceTransformer...")
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = embedder.encode(texts, show_progress_bar=True)

    k = 5  # Número de clusters (puedes ajustar este valor)
    print(f"Aplicando KMeans para {k} clusters...")
    kmeans = KMeans(n_clusters=k, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    df_posts["Cluster"] = clusters

    print("\nDistribución de clusters:")
    print(df_posts["Cluster"].value_counts())

    for cluster in range(k):
        print(f"\nEjemplos del Cluster {cluster}:")
        sample_texts = df_posts[df_posts["Cluster"] == cluster]["Texto"].head(3).tolist()
        for t in sample_texts:
            print("-", t[:200], "...")

    # --- Análisis de Tópicos con LDA ---
    print("\nAplicando análisis de tópicos con LDA...")
    vectorizer = TfidfVectorizer(stop_words='english', max_df=0.95, min_df=2)
    dtm = vectorizer.fit_transform(texts)
    lda = LatentDirichletAllocation(n_components=5, random_state=42)
    lda.fit(dtm)

    terms = vectorizer.get_feature_names_out()
    for idx, topic in enumerate(lda.components_):
        print(f"\nTópico #{idx}:")
        print([terms[i] for i in topic.argsort()[-10:]])

# Función principal para integrar análisis de sentimiento, preprocesamiento y análisis de tópicos/clustering
def main():
    df_posts_filtrados = obtener_posts_filtrados('climatechange', limite=100)

    if df_posts_filtrados is not None and not df_posts_filtrados.empty:
        # Convertir el DataFrame de Pandas a DataFrame de Spark
        spark_df = spark.createDataFrame(df_posts_filtrados)
        spark_df.printSchema()
        spark_df.show(5)

        # --- Preprocesamiento del texto con Spark ---
        tokenizer = Tokenizer(inputCol="Texto", outputCol="palabras_tokenizadas")
        df_tokenized = tokenizer.transform(spark_df)
        remover = StopWordsRemover(inputCol="palabras_tokenizadas", outputCol="palabras_limpiadas")
        df_clean = remover.transform(df_tokenized)
        df_clean.select("palabras_limpiadas").show(5, truncate=False)

        # --- Análisis de Sentimiento usando Hugging Face ---
        udf_analizar_sentimiento = udf(analizar_sentimiento_transformers, StringType())
        df_con_sentimiento = df_clean.withColumn("sentimiento", udf_analizar_sentimiento("Texto"))
        df_con_sentimiento.select("Texto", "sentimiento").show(5, truncate=False)

        # --- Análisis de Tópicos y Clustering ---
        print("\n--- Análisis de Clustering y Tópicos ---")
        analyze_clustering_and_topics(df_posts_filtrados)
    else:
        print("No se pudieron obtener los posts o el DataFrame está vacío.")

# Ejecutar la función principal
main()


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



root
 |-- Título: string (nullable = true)
 |-- Texto: string (nullable = true)
 |-- Fecha: timestamp (nullable = true)
 |-- Upvotes: long (nullable = true)
 |-- Número de Comentarios: long (nullable = true)
 |-- Enlace: string (nullable = true)

+--------------------+--------------------+-------------------+-------+---------------------+--------------------+
|              Título|               Texto|              Fecha|Upvotes|Número de Comentarios|              Enlace|
+--------------------+--------------------+-------------------+-------+---------------------+--------------------+
|Can this 'burnt t...|             No text|2025-02-23 20:29:06|     16|                    1|https://www.cbc.c...|
|During a snowy we...|             No text|2025-02-23 15:25:51|     22|                    0|https://www.nhpr....|
|Why Are We Focuse...|Every discussion ...|2025-02-23 03:59:40|      0|                   71|https://www.reddi...|
|Flying guilt and ...|As an average Ame...|2025-02-22 18:49:21|

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Aplicando KMeans para 5 clusters...

Distribución de clusters:
Cluster
1    14
3    12
4    11
2     6
0     2
Name: count, dtype: int64

Ejemplos del Cluster 0:
- Ive seen analysis that we are not going to go nonlinear,i.e. abrupt climate change, from llnl. Im wondering if that still holds. And im worried even gradual will still be sufficiently quick that we wi ...
- In James Hansen's last email newsletter he states the AMOC could shutdown mid-century, and then states "shutdown is irreversible in less than centuries."

My understanding is that the AMOC has a tippi ...

Ejemplos del Cluster 1:
- No text ...
- No text ...
- No text ...

Ejemplos del Cluster 2:
- As an average American who has started to see the effects of climate change in the past couple years, I can’t help but feel immense guilt and anxiety about everything.

I’ve already drastically reduce ...
- I wanna know some more obscure symptoms of chimate change (recent human-driven as well as cyclic historical). Some really i